In [ ]:
#!pip install pymysql
#!pip install mysql-connector

# **1. Introduction:**
- **Objective:** Bank GoodCredit wants to predict cred score for current credit card customers. The cred score will denote a customer’s credit worthiness and help the bank in reducing credit default risk.
- Target variable → Bad_label
    - 0 – Customer has Good credit history
    - 1 – Customer has Bad credit history

# **2. Importing Neccessary Libraries**

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import mysql.connector
import csv
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

In [ ]:
pd.set_option('display.max_columns',None)

# **3. Fetching data from Database**

In [ ]:
# Sql connection details
connection = mysql.connector.connect(
    host = '18.136.157.135',
    port = '3306',
    user  = 'dm_team1',
    password = 'DM!$Team&279@20!',
    database = 'project_banking'
    )

In [ ]:
def fetchresult(query):
    resultset = pd.read_sql_query(query,connection)
    return resultset

In [ ]:
query1 = 'show tables'
#print(fetchresult(query1))
query2 = 'select * from Cust_Account'
Cust_Account = fetchresult(query2)

In [ ]:
Cust_Account

,dt_opened,customer_no,upload_dt,acct_type,owner_indic,opened_dt,last_paymt_dt,closed_dt,reporting_dt,high_credit_amt,cur_balance_amt,amt_past_due,paymenthistory1,paymenthistory2,paymt_str_dt,paymt_end_dt,creditlimit,cashlimit,rateofinterest,paymentfrequency,actualpaymentamount
0,10-Nov-15,12265,20-Oct-15,6,1,09-Jun-13,30-Jun-14,05-Jul-14,30-Sep-15,20900,0,,"""""""STDSTDSTDXXXXXXXXXXXXXXXSTDXXXXXXXXXXXXXXXS...",,01-Sep-15,01-Jul-14,,,,,
1,10-Nov-15,12265,20-Oct-15,10,1,25-May-12,06-Sep-15,,03-Oct-15,16201,10390,,"""""""0000000000000000000000000000000000000000000...","""""""000000000000000000000000000XXX0000000000000...",01-Oct-15,01-Nov-12,14000,1400,,3,5603
2,10-Nov-15,12265,20-Oct-15,10,1,22-Mar-12,31-Aug-15,,30-Sep-15,41028,34420,,"""""""0000000000000000000000000000000000000000000...","""""""0000000000000000000000000000000000000000000...",01-Sep-15,01-Oct-12,,,,,
3,20-Jul-15,15606,09-Jul-15,10,1,13-Jan-06,,26-Jul-07,31-Jan-09,93473,0,,"""""""1200900600600600300000000000000000000000000...",,01-Jul-07,01-Feb-06,,,,,
4,20-Jul-15,15606,09-Jul-15,6,1,18-Jan-15,05-May-15,,31-May-15,20250,13500,,"""""""000000000000000""""""",,01-May-15,01-Jan-15,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186324,11-Jun-15,1611,25-May-15,10,1,02-May-07,21-Apr-15,,30-Apr-15,24228,24228,,"""""""0000000000000000000000000000000000000000000...","""""""0000000000000000000000000000000000000000000...",01-Apr-15,01-May-12,,,,,
186325,11-Jun-15,1611,25-May-15,10,1,11-Apr-07,22-Jun-09,02-Apr-10,30-Apr-15,19584,0,,"""""""000179146119089058029000XXX""""""",,01-Apr-10,01-Aug-09,,,,3,
186326,11-Jun-15,1611,25-May-15,10,1,20-Nov-06,,12-Apr-12,31-Dec-13,50844,0,,"""""""000XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...",,01-Apr-12,01-Dec-10,,,,3,
186327,11-Jun-15,1611,25-May-15,10,1,17-Oct-06,,07-Mar-12,15-Apr-15,35587,0,,"""""""000010000000000000000000000000000XXXXXXXXXX...","""""""XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...",01-Mar-12,01-Apr-09,15000,6000,,3,


# **4. EDA - Cust_Account**

In [ ]:
Cust_Account.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186329 entries, 0 to 186328
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   dt_opened            186329 non-null  object
 1   customer_no          186329 non-null  object
 2   upload_dt            186329 non-null  object
 3   acct_type            186329 non-null  object
 4   owner_indic          186329 non-null  object
 5   opened_dt            186329 non-null  object
 6   last_paymt_dt        186329 non-null  object
 7   closed_dt            186329 non-null  object
 8   reporting_dt         186329 non-null  object
 9   high_credit_amt      186329 non-null  object
 10  cur_balance_amt      186329 non-null  object
 11  amt_past_due         186329 non-null  object
 12  paymenthistory1      186329 non-null  object
 13  paymenthistory2      186329 non-null  object
 14  paymt_str_dt         186329 non-null  object
 15  paymt_end_dt         186329 non-nu

## **4.1 Cust_Account**

In [ ]:
Cust_Account.dt_opened = Cust_Account.dt_opened.astype('datetime64[ns]')

In [ ]:
Cust_Account.dt_opened.unique()

<DatetimeArray>
['2015-11-10 00:00:00', '2015-07-20 00:00:00', '2015-11-02 00:00:00',
 '2015-11-16 00:00:00', '2015-08-27 00:00:00', '2015-05-05 00:00:00',
 '2015-12-29 00:00:00', '2015-12-18 00:00:00', '2015-05-07 00:00:00',
 '2015-11-06 00:00:00',
 ...
 '2015-05-12 00:00:00', '2015-04-23 00:00:00', '2015-04-29 00:00:00',
 '2015-04-25 00:00:00', '2015-08-12 00:00:00', '2015-04-22 00:00:00',
 '2015-04-16 00:00:00', '2015-04-18 00:00:00', '2015-04-21 00:00:00',
 '2015-10-24 00:00:00']
Length: 197, dtype: datetime64[ns]

In [ ]:
Cust_Account.dt_opened.value_counts()

dt_opened
2015-11-16    4643
2015-12-21    2395
2015-08-18    2158
2015-10-13    2062
2015-10-26    2004
              ... 
2015-04-22      36
2015-04-18      18
2015-10-24       8
2015-04-16       2
2015-04-21       2
Name: count, Length: 197, dtype: int64

#### Inference:
- Date ranges from 16-Apr-2015 to 31-12-2015.
- Field is converted from object to datetime64[ns] type format.
- Total 196 unique dates are present.
- 16-Nov-2015 has max entries of 4643.
- Total entries 186329

## **4.2 customer_no**

In [ ]:
Cust_Account.customer_no.count()

186329

In [ ]:
len(Cust_Account.customer_no.unique())

23896

In [ ]:
Cust_Account.customer_no.value_counts()

customer_no
8516     120
10076    112
6391      88
8872      77
8814      76
        ... 
20548      1
21356      1
6941       1
362        1
23434      1
Name: count, Length: 23896, dtype: int64

In [ ]:
Cust_Account.customer_no = Cust_Account.customer_no.astype('int')

In [ ]:
Cust_Account.customer_no.max(),Cust_Account.customer_no.min()

(23896, 1)

### Inference:
- Total entries - 186329
- Unique customer numbers - 23896
- customer number 8516 has max entries of 120
- Customer number ranges from 1 to 23896
- Customer number field contains number. Hence converted to integer data type

## **4.3 upload_dt**

In [ ]:
Cust_Account.upload_dt = Cust_Account.upload_dt.astype('datetime64[ns]')

In [ ]:
Cust_Account.upload_dt.unique()

<DatetimeArray>
['2015-10-20 00:00:00', '2015-07-09 00:00:00', '2015-10-13 00:00:00',
 '2015-07-30 00:00:00', '2015-08-10 00:00:00', '2015-04-21 00:00:00',
 '2015-12-10 00:00:00', '2015-11-12 00:00:00', '2015-04-28 00:00:00',
 '2015-10-26 00:00:00', '2015-06-18 00:00:00', '2015-05-07 00:00:00',
 '2015-11-19 00:00:00', '2015-06-01 00:00:00', '2015-10-29 00:00:00',
 '2015-08-13 00:00:00', '2015-06-22 00:00:00', '2015-09-17 00:00:00',
 '2015-08-20 00:00:00', '2015-07-06 00:00:00', '2015-06-29 00:00:00',
 '2015-10-08 00:00:00', '2015-04-23 00:00:00', '2015-10-23 00:00:00',
 '2015-07-16 00:00:00', '2015-09-10 00:00:00', '2015-09-07 00:00:00',
 '2015-11-02 00:00:00', '2015-07-23 00:00:00', '2015-12-14 00:00:00',
 '2015-07-02 00:00:00', '2015-08-27 00:00:00', '2015-09-24 00:00:00',
 '2015-12-21 00:00:00', '2015-09-04 00:00:00', '2015-10-15 00:00:00',
 '2015-07-20 00:00:00', '2015-11-16 00:00:00', '2015-08-26 00:00:00',
 '2015-10-01 00:00:00', '2015-05-18 00:00:00', '2015-06-25 00:00:00',
 '20

In [ ]:
Cust_Account.upload_dt.value_counts()

upload_dt
2015-04-21    5128
2015-10-23    3591
2015-07-16    3512
2015-09-28    3507
2015-10-20    3486
              ... 
2015-12-17    1154
2015-12-24     334
2015-12-28     106
2015-12-31      53
2015-11-29      36
Name: count, Length: 75, dtype: int64

#### Inference:
- upload date ranges from 21-Apr-2015 to 31-12-2015.
- Field is converted from object to datetime64[ns] type format.
- Total 75 unique dates are present.
- 21-Apr-2015 has max entries of 5128.
- Total entries 186329

## **4.4 acct_type**

In [ ]:
Cust_Account.acct_type.unique()

array(['6', '10', '1', '13', '0', '5', '7', '12', '51', '2', '3', '32',
       '15', '8', '52', '4', '56', '17', '53', '54', '35', '9', '34',
       '11', '59', '33', '14', '31', '55', '16', '57'], dtype=object)

In [ ]:
len(Cust_Account.acct_type.unique())

31

In [ ]:
Cust_Account.acct_type.value_counts()

acct_type
10    100239
6      25664
5      22921
1       9427
13      8569
2       5639
7       4256
0       3944
12      1196
32       744
51       732
3        672
15       519
17       453
8        407
53       386
4        188
52        97
54        73
35        54
9         53
31        27
59        22
55        17
56         8
34         5
33         5
14         5
11         4
16         2
57         1
Name: count, dtype: int64

In [ ]:
# Since all the value in this column are numbers, converting the field data type to integer.
Cust_Account.acct_type = Cust_Account.acct_type.astype('int')

In [ ]:
Cust_Account.acct_type.min(),Cust_Account.acct_type.max()

(0, 59)

#### Inference:
- acct_type field tells the type of account user is holding(i.e Savings, current etc)
- Total 31 unique accounts are present
- acct_type ranges from 0 to 59.
- With acct_type 10 holds max of 100239 entries and acct_type 57 holds 1 entries.
- Field converted to integer type.

## **4.5 owner_indic**

In [ ]:
Cust_Account.owner_indic.unique()

array(['1', '4', '2', '3'], dtype=object)

In [ ]:
Cust_Account.owner_indic.value_counts()

owner_indic
1    177287
4      6253
2      1581
3      1208
Name: count, dtype: int64

#### Inference:
- owner_indic contains 4 unique values.
- Value 1 contains max of 177287 entries.
- Since purpose of this column is unknown, column can be skipped.

## **4.6 opened_dt**

In [ ]:
Cust_Account.opened_dt = Cust_Account.opened_dt.astype('datetime64[ns]')

In [ ]:
Cust_Account.opened_dt.value_counts()

opened_dt
2012-04-13    514
2014-10-31    472
2014-12-31    341
2015-01-31    341
2014-09-30    335
             ... 
1998-09-24      1
2015-10-02      1
2001-05-01      1
1984-08-01      1
2000-06-02      1
Name: count, Length: 6245, dtype: int64

In [ ]:
Cust_Account.dropna(inplace=True)

### Inference:
- 6246 unique dates are present
- 13-Apr-2015 has max entries of 514
- 426 null values present in the data.
- 426 rows removed.
- Field is converted to datetime64[ns] data type.

## **4.7 last_paymt_dt**

In [ ]:
Cust_Account.last_paymt_dt = Cust_Account.last_paymt_dt.astype('datetime64[ns]')

In [ ]:
Cust_Account.last_paymt_dt.unique()

<DatetimeArray>
['2014-06-30 00:00:00', '2015-09-06 00:00:00', '2015-08-31 00:00:00',
                 'NaT', '2015-05-05 00:00:00', '2015-05-02 00:00:00',
 '2015-05-08 00:00:00', '2015-05-21 00:00:00', '2011-12-03 00:00:00',
 '2011-10-27 00:00:00',
 ...
 '2006-11-05 00:00:00', '2003-06-11 00:00:00', '2004-07-30 00:00:00',
 '1998-01-03 00:00:00', '2004-11-26 00:00:00', '2005-05-04 00:00:00',
 '2002-11-16 00:00:00', '2007-07-13 00:00:00', '2002-05-04 00:00:00',
 '2002-07-16 00:00:00']
Length: 4511, dtype: datetime64[ns]

In [ ]:
Cust_Account.last_paymt_dt.value_counts()

last_paymt_dt
2015-03-05    1739
2015-09-05    1516
2015-08-05    1323
2015-05-05    1283
2015-07-02    1201
              ... 
1999-12-28       1
2004-09-03       1
2004-10-06       1
2011-02-06       1
2002-07-16       1
Name: count, Length: 4510, dtype: int64

In [ ]:
Cust_Account[Cust_Account.last_paymt_dt.isnull()]

,dt_opened,customer_no,upload_dt,acct_type,owner_indic,opened_dt,last_paymt_dt,closed_dt,reporting_dt,high_credit_amt,cur_balance_amt,amt_past_due,paymenthistory1,paymenthistory2,paymt_str_dt,paymt_end_dt,creditlimit,cashlimit,rateofinterest,paymentfrequency,actualpaymentamount
3,2015-07-20,15606,2015-07-09,10,1,2006-01-13,NaT,26-Jul-07,31-Jan-09,93473,0,,"""""""1200900600600600300000000000000000000000000...",,01-Jul-07,01-Feb-06,,,,,
15,2015-07-20,15606,2015-07-09,10,1,2007-01-19,NaT,21-Mar-07,31-May-11,29105,0,,"""""""XXX""""""",,01-Mar-07,01-Mar-07,,,,,
17,2015-11-02,11742,2015-10-13,10,1,2015-08-27,NaT,,31-Aug-15,,0,,"""""""000""""""",,01-Aug-15,01-Aug-15,,,,,
18,2015-11-02,11742,2015-10-13,0,1,2015-08-19,NaT,,30-Sep-15,20000,14000,,"""""""000000""""""",,01-Sep-15,01-Aug-15,,,,,
19,2015-11-02,11742,2015-10-13,5,1,2015-08-08,NaT,,31-Aug-15,149000,149000,,"""""""000""""""",,01-Aug-15,01-Aug-15,,,,3,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186286,2015-12-17,13985,2015-09-07,5,1,2015-07-21,NaT,,31-Jul-15,200000,200000,,"""""""000""""""",,01-Jul-15,01-Jul-15,,,,,
186314,2015-08-07,16778,2015-07-27,10,1,2006-12-26,NaT,27-Oct-14,16-Jun-15,67044,0,,"""""""0000000000000000000000000000000000000000000...",,01-Oct-14,01-May-13,199000,79600,,3,
186326,2015-06-11,1611,2015-05-25,10,1,2006-11-20,NaT,12-Apr-12,31-Dec-13,50844,0,,"""""""000XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...",,01-Apr-12,01-Dec-10,,,,3,
186327,2015-06-11,1611,2015-05-25,10,1,2006-10-17,NaT,07-Mar-12,15-Apr-15,35587,0,,"""""""000010000000000000000000000000000XXXXXXXXXX...","""""""XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...",01-Mar-12,01-Apr-09,15000,6000,,3,


### Inference:
- 4511 unique dates present in this field.
- 05-Mar-2015 has max entries of 1739 & 04-May-2002 has least entries of 1.
- 21 null values present in the data.
- Field is converted to datetime64[ns] data type.

## **4.8 closed_dt**

In [ ]:
Cust_Account.closed_dt = Cust_Account.closed_dt.astype('str')

In [ ]:
Cust_Account.closed_dt.value_counts()

closed_dt
             108774
05-Mar-12       353
11-Aug-11       291
31-Dec-08       283
05-Mar-13       282
              ...  
17-Mar-03         1
04-Sep-03         1
28-Aug-00         1
02-Apr-04         1
01-Aug-03         1
Name: count, Length: 4840, dtype: int64

### Inference:
- 107715 null values present in the data.
- Column can be dropped.

## **4.9 reporting_dt**

In [ ]:
Cust_Account.reporting_dt = Cust_Account.reporting_dt.astype('datetime64[ns]')

In [ ]:
Cust_Account.reporting_dt.unique()

<DatetimeArray>
['2015-09-30 00:00:00', '2015-10-03 00:00:00', '2009-01-31 00:00:00',
 '2015-05-31 00:00:00', '2015-06-26 00:00:00', '2011-12-31 00:00:00',
 '2014-02-28 00:00:00', '2013-12-31 00:00:00', '2011-06-30 00:00:00',
 '2014-11-30 00:00:00',
 ...
 '2010-06-24 00:00:00', '2011-04-08 00:00:00', '2014-09-17 00:00:00',
 '2013-03-05 00:00:00', '2009-09-01 00:00:00', '2010-09-08 00:00:00',
 '2006-11-15 00:00:00', '2014-05-27 00:00:00', '2012-04-24 00:00:00',
 '2010-11-07 00:00:00']
Length: 1872, dtype: datetime64[ns]

In [ ]:
Cust_Account.reporting_dt.value_counts()

reporting_dt
2015-07-31    12618
2015-08-31    10119
2015-09-30     9962
2015-05-31     9772
2015-06-30     9363
              ...  
2009-04-29        1
2012-05-15        1
2011-08-17        1
2013-01-17        1
2010-11-07        1
Name: count, Length: 1872, dtype: int64

In [ ]:
Cust_Account[Cust_Account.reporting_dt.isnull()]

,dt_opened,customer_no,upload_dt,acct_type,owner_indic,opened_dt,last_paymt_dt,closed_dt,reporting_dt,high_credit_amt,cur_balance_amt,amt_past_due,paymenthistory1,paymenthistory2,paymt_str_dt,paymt_end_dt,creditlimit,cashlimit,rateofinterest,paymentfrequency,actualpaymentamount


### Inference:
- 1877 unique dates present in this field.
- 31-Jul-2015 has max entries of 12654 & 07-Nov-2010 has least entries of 1.
- No null values present in the data.
- Field is converted to datetime64[ns] data type.

## **4.10 high_credit_amt**

In [ ]:
# to detect null values converting to str and checking
Cust_Account.high_credit_amt = Cust_Account.high_credit_amt.astype('str')

In [ ]:
Cust_Account.loc[Cust_Account.high_credit_amt == '','high_credit_amt'] = '0'

In [ ]:
Cust_Account.high_credit_amt = Cust_Account.high_credit_amt.astype('int')

In [ ]:
# Handling null/empty values
Cust_Account.loc[Cust_Account.high_credit_amt == 0,'high_credit_amt'] = Cust_Account.high_credit_amt.median()

In [ ]:
len(Cust_Account.high_credit_amt.unique())

67526

In [ ]:
Cust_Account.high_credit_amt.value_counts()

high_credit_amt
42250     8654
100000    3969
150000    2311
200000    2173
300000    1626
          ... 
306658       1
2019         1
48929        1
10461        1
29516        1
Name: count, Length: 67526, dtype: int64

In [ ]:
Cust_Account.high_credit_amt.min(),Cust_Account.high_credit_amt.max()

(1, 180000000)

### Inference:
- 67554 unique transaction amounts present in this field.
- 1 is the min txn amount and max transaction amount is 18,00,00,000 (18 crores)
- Transaction amount 42122 done 8877 times by different users.
- Null values are replaced with median values.
- Field is converted to int data type.

## **4.11 cur_balance_amt**

In [ ]:
Cust_Account.cur_balance_amt = Cust_Account.cur_balance_amt.astype('int')

In [ ]:
len(Cust_Account.cur_balance_amt.unique())

56359

In [ ]:
Cust_Account.cur_balance_amt.value_counts()

cur_balance_amt
 0         89601
-1          1918
-2           434
-3           307
-4           228
           ...  
 166812        1
 31197         1
 41091         1
 399084        1
 24228         1
Name: count, Length: 56359, dtype: int64

In [ ]:
Cust_Account.loc[Cust_Account.cur_balance_amt < 0,'cur_balance_amt']

39         -34
42          -7
54          -2
65          -1
72          -1
          ... 
186234      -3
186251   -1463
186275      -7
186300     -80
186306      -7
Name: cur_balance_amt, Length: 8878, dtype: int32

In [ ]:
Cust_Account.loc[Cust_Account.cur_balance_amt == 0,'cur_balance_amt']

0         0
3         0
10        0
11        0
12        0
         ..
186323    0
186325    0
186326    0
186327    0
186328    0
Name: cur_balance_amt, Length: 89601, dtype: int32

In [ ]:
Cust_Account.loc[Cust_Account.cur_balance_amt > 0,'cur_balance_amt']

1          10390
2          34420
4          13500
5           8000
6          12550
           ...  
186301     19013
186302     10737
186309     38185
186318    287017
186324     24228
Name: cur_balance_amt, Length: 87395, dtype: int32

In [ ]:
Cust_Account.cur_balance_amt.min(),Cust_Account.cur_balance_amt.max()

(-7355205, 136009965)

### Inference:
- This field refers to customers current account balance.
- 89872 customers have zero balance
- 8881 customers have negative balance
- 1 user having highest negative balance of '7355205'
- 1 user having highest current balance of '136009965'

## **4.12 amt_past_due**

In [ ]:
len(Cust_Account.amt_past_due.unique())

630

In [ ]:
Cust_Account.amt_past_due.value_counts()

amt_past_due
        185000
400         43
2           30
1           25
3           18
         ...  
89           1
2459         1
504          1
5129         1
8769         1
Name: count, Length: 630, dtype: int64

### Inference:
- This field has most values null/empty.
- Hence this field can be removed

## **4.13 paymenthistory1 & paymenthistory2**

### Inference:
- Both the fields can be removed.

## **4.14 paymt_str_dt**

In [ ]:
Cust_Account.paymt_str_dt = Cust_Account.paymt_str_dt.astype('datetime64[ns]')

In [ ]:
Cust_Account.loc[Cust_Account.paymt_str_dt.isnull(),'paymt_str_dt']

2759   NaT
Name: paymt_str_dt, dtype: datetime64[ns]

In [ ]:
Cust_Account.dropna(subset=['paymt_str_dt'], inplace=True)

In [ ]:
Cust_Account.paymt_str_dt.unique()

<DatetimeArray>
['2015-09-01 00:00:00', '2015-10-01 00:00:00', '2007-07-01 00:00:00',
 '2015-05-01 00:00:00', '2015-06-01 00:00:00', '2011-12-01 00:00:00',
 '2011-11-01 00:00:00', '2010-10-01 00:00:00', '2011-06-01 00:00:00',
 '2014-04-01 00:00:00',
 ...
 '1997-05-01 00:00:00', '2000-07-01 00:00:00', '1997-06-01 00:00:00',
 '1996-06-01 00:00:00', '1996-03-01 00:00:00', '1997-01-01 00:00:00',
 '1998-06-01 00:00:00', '2000-10-01 00:00:00', '1995-09-01 00:00:00',
 '1997-10-01 00:00:00']
Length: 233, dtype: datetime64[ns]

## Inference:
- 1 Null value dropped

## **4.15 paymt_end_dt**

In [ ]:
Cust_Account.paymt_end_dt = Cust_Account.paymt_end_dt.astype('datetime64[ns]')

In [ ]:
Cust_Account.paymt_end_dt.unique()

<DatetimeArray>
['2014-07-01 00:00:00', '2012-11-01 00:00:00', '2012-10-01 00:00:00',
 '2006-02-01 00:00:00', '2015-01-01 00:00:00', '2014-12-01 00:00:00',
 '2014-10-01 00:00:00', '2012-12-01 00:00:00', '2012-08-01 00:00:00',
 '2009-01-01 00:00:00',
 ...
 '2000-07-01 00:00:00', '1997-06-01 00:00:00', '1996-06-01 00:00:00',
 '1996-03-01 00:00:00', '1997-01-01 00:00:00', '1998-06-01 00:00:00',
 '2000-10-01 00:00:00', '1993-02-01 00:00:00', '1995-09-01 00:00:00',
 '1997-10-01 00:00:00']
Length: 234, dtype: datetime64[ns]

In [ ]:
Cust_Account.paymt_end_dt.value_counts()

paymt_end_dt
2015-01-01    5634
2012-10-01    4982
2012-09-01    4740
2014-10-01    4691
2012-07-01    4611
              ... 
1996-01-01       1
1993-10-01       1
1993-02-01       1
1995-09-01       1
1997-10-01       1
Name: count, Length: 234, dtype: int64

In [ ]:
Cust_Account.dropna(subset=['paymt_end_dt'],inplace=True)

## **4.16 creditlimit**

In [ ]:
len(Cust_Account.creditlimit.unique())

1380

In [ ]:
Cust_Account.creditlimit.value_counts()

creditlimit
          137039
50000       2558
10000       1966
20000       1193
15000       1126
           ...  
15190          1
63200          1
254600         1
323000         1
42490          1
Name: count, Length: 1380, dtype: int64

### Inference:
- 137476 null vales are present
- Column can be dropped since majority having null value

## **4.17 cashlimit**

In [ ]:
Cust_Account.cashlimit.value_counts()

cashlimit
          150595
10000       3827
5000         717
1000         651
6000         584
           ...  
48650          1
28100          1
22903          1
193200         1
11310          1
Name: count, Length: 1456, dtype: int64

### Inference:
- 151046 null vales are present
- Column can be dropped since majority having null value

## **4.18 rateofinterest**

In [ ]:
Cust_Account.rateofinterest.value_counts()

rateofinterest
          161195
39          5025
40.8        2520
41.4        1883
10          1494
           ...  
27.67          1
27.045         1
20.11          1
7.54           1
16.402         1
Name: count, Length: 1416, dtype: int64

### Inference:
- 161495 null vales are present
- Column can be dropped since majority having null value

## **4.19 paymentfrequency**

In [ ]:
Cust_Account.paymentfrequency.value_counts()

paymentfrequency
     122177
3     63575
1       121
Name: count, dtype: int64

### Inference:
- 122435 null vales are present
- Column can be dropped since majority having null value

## **4.20 actualpaymentamount**

In [ ]:
Cust_Account.actualpaymentamount.value_counts()

actualpaymentamount
          144824
5000         507
10000        448
2000         362
3000         312
           ...  
24917          1
23433          1
119864         1
143424         1
39499          1
Name: count, Length: 18299, dtype: int64

### Inference:
- 145275 null vales are present
- Column can be dropped since majority having null value

## **4.21 Removing unwanted columns**

In [ ]:
drop_columns = ['closed_dt','owner_indic','last_paymt_dt','amt_past_due','paymenthistory1','paymenthistory2','creditlimit', 'cashlimit', 'rateofinterest', 'paymentfrequency','actualpaymentamount']
Cust_Account.drop(drop_columns,axis=1,inplace=True)

## **4.22 Removing Duplicated Rows**

In [ ]:
Cust_Account.drop_duplicates(inplace=True)

## **4.23 Reducing dataset memory**

In [ ]:
for column_name in Cust_Account:
    if Cust_Account[column_name].dtype == 'int32':
        Cust_Account[column_name] = pd.to_numeric(Cust_Account[column_name], downcast='integer')

In [ ]:
Cust_Account.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183113 entries, 0 to 186328
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   dt_opened        183113 non-null  datetime64[ns]
 1   customer_no      183113 non-null  int16         
 2   upload_dt        183113 non-null  datetime64[ns]
 3   acct_type        183113 non-null  int8          
 4   opened_dt        183113 non-null  datetime64[ns]
 5   reporting_dt     183113 non-null  datetime64[ns]
 6   high_credit_amt  183113 non-null  int32         
 7   cur_balance_amt  183113 non-null  int32         
 8   paymt_str_dt     183113 non-null  datetime64[ns]
 9   paymt_end_dt     183113 non-null  datetime64[ns]
dtypes: datetime64[ns](6), int16(1), int32(2), int8(1)
memory usage: 11.7 MB


- In the original dataset, memory consumed is 30 MB. Processing time will be more if we process with this huge datset size.So before going into processing, we are reducing the memory consumed by the dataset.
- Before reducing -30 MB
- After reducing - 11.7 MB

In [ ]:
Cust_Account

,dt_opened,customer_no,upload_dt,acct_type,opened_dt,reporting_dt,high_credit_amt,cur_balance_amt,paymt_str_dt,paymt_end_dt
0,2015-11-10,12265,2015-10-20,6,2013-06-09,2015-09-30,20900,0,2015-09-01,2014-07-01
1,2015-11-10,12265,2015-10-20,10,2012-05-25,2015-10-03,16201,10390,2015-10-01,2012-11-01
2,2015-11-10,12265,2015-10-20,10,2012-03-22,2015-09-30,41028,34420,2015-09-01,2012-10-01
3,2015-07-20,15606,2015-07-09,10,2006-01-13,2009-01-31,93473,0,2007-07-01,2006-02-01
4,2015-07-20,15606,2015-07-09,6,2015-01-18,2015-05-31,20250,13500,2015-05-01,2015-01-01
...,...,...,...,...,...,...,...,...,...,...
186324,2015-06-11,1611,2015-05-25,10,2007-05-02,2015-04-30,24228,24228,2015-04-01,2012-05-01
186325,2015-06-11,1611,2015-05-25,10,2007-04-11,2015-04-30,19584,0,2010-04-01,2009-08-01
186326,2015-06-11,1611,2015-05-25,10,2006-11-20,2013-12-31,50844,0,2012-04-01,2010-12-01
186327,2015-06-11,1611,2015-05-25,10,2006-10-17,2015-04-15,35587,0,2012-03-01,2009-04-01


In [ ]:
file_name = 'Cust_Account_processed.csv'
Cust_Account.to_csv(file_name,index=False)

# **5. EDA - Cust_Demographics**

In [ ]:
query3 = 'select * from Cust_Demographics'
Cust_Demographics = fetchresult(query3)

In [ ]:
Cust_Demographics

,dt_opened,customer_no,entry_time,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,Bad_label
0,18-Apr-15,1,13-Apr-15,Insignia,13-Apr-15,650,2,Card Setup,14,500000,,,,Y,IS1,,0,159,4284,4284,,1,ADFPNXXXXX,03-Sep-65,98332XXXXX,N,@REDIFFMAIL.COM,1,2,,Mumbai / Navi Mumbai / Thane,400610,1965,0,Self,Y,2,90000,,,,0,0,0,0,Mumbai,400059,@CODOGNOTTO.NET,PAN Card,ADFPNXXXXX,The Ratnakar Bank Ltd.,,Y,State Bank of India,0,01-Jun-13,17-Jun-16,1,21,,Y,Y,N,,Y,1965-0,21,15,400610,0,2,90000,Nov-00,21,R,,,0000-00-00,0,98332XXXXX,1,N,0
1,21-Apr-15,2,21-Apr-15,Insignia,21-Apr-15,760,1,Card Setup,14,1200000,,,,Y,IS1,,0,91,B001,4077,,1,AJWPRXXXXX,14-Jul-62,99455XXXXX,N,@GMAIL.COM,1,2,,Bengaluru,560042,1969,0,Self,Y,2,1,,,,0,0,0,0,Bangalore,560042,,PAN Card,AJWPRXXXXX,The Ratnakar Bank Ltd.,,N,,0,,17-Jun-16,1,17,,Y,Y,N,,Y,1969-0,17,12,560042,0,2,1,Nov-00,17,R,,,0000-00-00,0,99455XXXXX,1,N,0
2,22-Apr-15,3,21-Apr-15,Insignia,21-Apr-15,774,1,Card Setup,14,700000,,,,Y,IS1,,0,91,B001,4077,,2,AFAPNXXXXX,10-Apr-66,98456XXXXX,N,@SHOBANARAYAN.COM,1,0,,Bengaluru,560042,1966,0,Self,Y,2,1,,,,0,0,0,0,Bangalore,560042,,PAN Card,AFAPNXXXXX,,,N,,0,,17-Jun-16,3,17,,N,Y,N,,Y,1966-0,17,12,560042,0,2,1,Nov-00,17,R,,,0000-00-00,0,98456XXXXX,1,N,0
3,25-Apr-15,4,15-Apr-15,Insignia,20-Apr-15,770,1,Card Setup,14,500000,,,,Y,IS1,,0,157,5107,5107,,1,AAAPDXXXXX,16-Apr-64,98220XXXXX,N,@VSNL.COM,1,3,,Pune,411001,1988,0,Self,Y,2,100000,,,,0,0,0,0,Pune,411026,@ALBAJ.COM,PAN Card,AAAPDXXXXX,The Ratnakar Bank Ltd.,,Y,HDFC Bank,0,,17-Jun-16,1,21,,Y,Y,N,,Y,1988-0,21,16,411001,0,2,100000,Nov-00,21,R,,,6/15/65,1,98220XXXXX,1,N,0
4,06-May-15,5,30-Apr-15,Insignia,,,3,Card Setup,14,500000,,,,Y,IS1,,0,100,D016,4564,,1,ABEPSXXXXX,03-Jan-54,98111XXXXX,N,@REDIFFMAIL.COM,1,3,,Gurgaon,122009,1995,0,Self,Y,2,300000,,,,0,0,0,0,Gurgaon,122009,,PAN Card,ABEPSXXXXX,The Ratnakar Bank Ltd.,,N,,0,,17-Jun-16,1,13,,Y,Y,N,,Y,1995-0,13,3,122009,0,2,300000,Nov-00,13,R,,,0000-00-00,0,98111XXXXX,1,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23891,31-Dec-15,23892,10-Dec-15,Platinum Deligh,10-Dec-15,704,1,Card Setup,14,119000,,,,Y,PD1,PD3,12,SA05,MA01,,,1,CUQPSXXXXX,12-Oct-88,98715XXXXX,N,@YAHOO.COM,2,0,Graduate,New Delhi,110045,1988,10,Paren,Y,1,33000,Private Ltd. Co.,Others,SERVICE ASSOCIATE,2015,6,0,6,Gurgaon,122001,,PAN Card,CUQPSXXXXX,,,Y,State Bank of India,99000,01-Jan-15,17-Jun-16,1,10,Y,N,Y,N,,Y,1988-10,10,157,110045,0,1,33000,0-0,13,R,Y,,0000-00-00,0,XXXXX,1,N,0
23892,17-Dec-15,23893,16-Nov-15,Platinum Deligh,,,2,Card Setup,14,35000,,,,Y,PD1,PD3,12,SA03,AS33,,,1,DTPPSXXXXX,01-Feb-87,99101XXXXX,N,@GMAIL.COM,2,0,Graduate,New Delhi,110037,2014,12,Rente,N,1,1,Private Ltd. Co.,Others,ENGINEER,2012,12,3,0,New Delhi,110037,@GMAIL.COM,PAN Card,DTPPSXXXXX,,,N,,0,,17-Jun-16,1,10,N,N,Y,N,,Y,2014-12,33,102,282001,1,1,1,0-0,10,O,Y,,0000-00-00,0,99101XXXXX,1,N,0
23893,04-Nov-15,23894,24-Sep-15,Platinum Maxima,24-Sep-15,748,3,Card 

In [ ]:
Cust_Demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23896 entries, 0 to 23895
Data columns (total 83 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   dt_opened    23896 non-null  object
 1   customer_no  23896 non-null  object
 2   entry_time   23896 non-null  object
 3   feature_1    23896 non-null  object
 4   feature_2    23896 non-null  object
 5   feature_3    23896 non-null  object
 6   feature_4    23896 non-null  object
 7   feature_5    23896 non-null  object
 8   feature_6    23896 non-null  object
 9   feature_7    23896 non-null  object
 10  feature_8    23896 non-null  object
 11  feature_9    23896 non-null  object
 12  feature_10   23896 non-null  object
 13  feature_11   23896 non-null  object
 14  feature_12   23896 non-null  object
 15  feature_13   23896 non-null  object
 16  feature_14   23896 non-null  object
 17  feature_15   23896 non-null  object
 18  feature_16   23896 non-null  object
 19  feature_17   23896 non-nu

## **5.1 dt_opened**

In [ ]:
Cust_Demographics.dt_opened = Cust_Demographics.dt_opened.astype('datetime64[ns]')

In [ ]:
Cust_Demographics.dt_opened.unique()

<DatetimeArray>
['2015-04-18 00:00:00', '2015-04-21 00:00:00', '2015-04-22 00:00:00',
 '2015-04-25 00:00:00', '2015-05-06 00:00:00', '2015-05-12 00:00:00',
 '2015-05-22 00:00:00', '2015-05-28 00:00:00', '2015-05-30 00:00:00',
 '2015-06-01 00:00:00',
 ...
 '2015-12-14 00:00:00', '2015-12-16 00:00:00', '2015-12-18 00:00:00',
 '2015-12-23 00:00:00', '2015-12-26 00:00:00', '2015-12-28 00:00:00',
 '2015-12-29 00:00:00', '2015-12-31 00:00:00', '2015-08-12 00:00:00',
 '2015-10-24 00:00:00']
Length: 197, dtype: datetime64[ns]

In [ ]:
Cust_Demographics.dt_opened.value_counts()

dt_opened
2015-11-16    699
2015-12-21    324
2015-10-13    279
2015-10-26    271
2015-11-24    268
             ... 
2015-04-22      4
2015-10-24      3
2015-04-21      1
2015-04-16      1
2015-04-18      1
Name: count, Length: 197, dtype: int64

## **5.2 customer_no**

In [ ]:
Cust_Demographics.customer_no = Cust_Demographics.customer_no.astype('int')

In [ ]:
Cust_Demographics.customer_no.value_counts()

customer_no
1        1
15929    1
15938    1
15937    1
15936    1
        ..
7963     1
7962     1
7961     1
7960     1
23896    1
Name: count, Length: 23896, dtype: int64

In [ ]:
Cust_Demographics.customer_no.min(),Cust_Demographics.customer_no.max()

(1, 23896)

### Inference:
- No null value present
- Total unique value - 23896

## **5.3 entry_time**

In [ ]:
Cust_Demographics.entry_time = Cust_Demographics.entry_time.astype('datetime64[ns]')

In [ ]:
Cust_Demographics.entry_time.value_counts()

entry_time
2015-10-19    180
2015-09-11    180
2015-10-23    174
2015-10-09    172
2015-08-07    172
             ... 
2015-04-05      1
2015-12-20      1
2015-04-03      1
2015-03-07      1
2015-03-17      1
Name: count, Length: 296, dtype: int64

In [ ]:
Cust_Demographics.loc[Cust_Demographics.entry_time.isnull()]

,dt_opened,customer_no,entry_time,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,Bad_label
1566,2015-06-10,1567,NaT,,,,,,,,,,,,,,,,,,,,XXXXX,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,0
2231,2015-06-27,2232,NaT,,,,,,,,,,,,,,,,,,,,XXXXX,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,0
7250,2015-06-27,7251,NaT,,,,,,,,,,,,,,,,,,,,XXXXX,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,0
8687,2015-09-02,8688,NaT,,,,,,,,,,,,,,,,,,,,XXXXX,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,0
8807,2015-09-03,8808,NaT,,,,,,,,,,,,,,,,,,,,XXXXX,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,0
12048,2015-11-06,12049,NaT,,,,,,,,,,,,,,,,,,,,XXXXX,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,0
12250,2015-11-10,12251,NaT,,,,,,,,,,,,,,,,,,,,XXXXX,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,0
12581,2015-11-17,12582,NaT,,,,,,,,,,,,,,,,,,,,XXXXX,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,0
12590,2015-11-18,12591,NaT,,,,,,,,,,,,,,,,,,,,XXXXX,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,0
13333,2015-12-03,13334,NaT,,,,,,,,,,,,,,,,,,,,XXXXX,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,XXXXX,,,1


In [ ]:
# 15 records are almost empty for most columns. hence deleting before processing.
drop_rows = Cust_Demographics.loc[Cust_Demographics.entry_time.isnull()].index
Cust_Demographics.drop(drop_rows,axis=0,inplace=True)

## **5.4 Bad_label**

In [ ]:
Cust_Demographics.Bad_label = Cust_Demographics.Bad_label.astype('int')

In [ ]:
Cust_Demographics[Cust_Demographics.Bad_label.isnull()]

,dt_opened,customer_no,entry_time,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,Bad_label


In [ ]:
Cust_Demographics.Bad_label.value_counts()

Bad_label
0    22878
1     1003
Name: count, dtype: int64

#### Inference:
- This is target variable.
- Field converted to integer type.
- 0 - Customer has Good credit history
- 1- Customer has bad credit history
- Majority of the customers have good credit history.

## **5.5 feature_1 to feature_79**

In [ ]:
# Identifying the null values in each column
text = 'feature_'
counts = []

for i in range(1,79):
    feature = text + str(i)
    count = len(Cust_Demographics[Cust_Demographics[feature] == ''])
    counts.append((feature, count))

# Display the counts
for feature, count in counts:
    print(f'{feature}: {count}')

feature_1: 0
feature_2: 2821
feature_3: 2821
feature_4: 0
feature_5: 0
feature_6: 0
feature_7: 0
feature_8: 22620
feature_9: 22620
feature_10: 23830
feature_11: 0
feature_12: 0
feature_13: 12989
feature_14: 7718
feature_15: 8
feature_16: 12
feature_17: 22854
feature_18: 23863
feature_19: 0
feature_20: 0
feature_21: 0
feature_22: 0
feature_23: 0
feature_24: 625
feature_25: 0
feature_26: 0
feature_27: 3622
feature_28: 0
feature_29: 0
feature_30: 0
feature_31: 0
feature_32: 0
feature_33: 0
feature_34: 0
feature_35: 0
feature_36: 5667
feature_37: 5667
feature_38: 5667
feature_39: 0
feature_40: 0
feature_41: 0
feature_42: 0
feature_43: 0
feature_44: 0
feature_45: 13698
feature_46: 79
feature_47: 0
feature_48: 18384
feature_49: 23777
feature_50: 0
feature_51: 11407
feature_52: 0
feature_53: 11595
feature_54: 0
feature_55: 0
feature_56: 0
feature_57: 21488
feature_58: 0
feature_59: 0
feature_60: 0
feature_61: 23872
feature_62: 0
feature_63: 0
feature_64: 0
feature_65: 0
feature_66: 0
feature_

### Inference:
- Feature_1 - only 15 records are null
- Feature_2 - 2837 null values. Its date field. Null values cannot be replaced and to be dropped
- Feature_3 - 2837 null values. Most of the values are integers and null value can be replaced with '0'. Field can be retained.
- Feature_4 - 2837 null values. No info about the field. 1,2,3 unique. can be dropped
- Feature_5 - only 15 records are null. Others are Card Setup. Dropped
- Feature_6 - only 15 records are null. Others are 14. Dropped
- Feature_7 - only 15 records are null. Retain
- Feature_8 - 22636 null values. Replace Null value with 'Not Applicable'. Retain
- Feature_9 - 22621 null values. Replace Null value with 'Not Applicable'. Retain
- Feature_10 - 23831 null values. Dropped
- Feature_11 - only 15 records are null. Retain. One hot encoder can be used to convert cat value to numerical value.
- Feature_12 - only 15 records are null. Retain
- Feature_13 - 13005 records empty. Need to decide
- Feature_14 - 7734 records empty. 0 & 12 are other values. Need to decide
- Feature_15 - 24 records empty. contains numeric & characters as values. Difficult to encode. Can be dropped
- Feature_16 - 28 records empty. contains numeric & characters as values. Difficult to encode. Can be dropped
- Feature_17 - 22870 records are empty. Can be dropped
- Feature_18 - 23879 records are empty. Can be dropped
- Feature_19 - only 15 records are null. Retain
- Feature_20 - No null values. Seems to be masked value of PAN. can be dropped.
- Feature_21 - only 15 records are null. Retain
- Feature_22 - Masked mobile no. No use. Dropped
- Feature_23 - only 15 records are null. Retain. One hot encoder can be used to convert cat value to numerical value.
- Feature_24 - Email address. Not in full address. can be dropped
- Feature_25 - only 15 records are null. Retain
- Feature_26 - only 15 records are null. 0-10 values present. Retain
- Feature_27 - 3638 records are null. Need to decide
- Feature_28 - only 15 records are null. Retain
- Feature_29 - only 15 records are null. Retain
- Feature_30 - only 15 records are null. Retain
- Feature_31 - only 15 records are null. Retain
- Feature_32 - only 15 records are null. Retain
- Feature_33 - only 15 records are null. Retain
- Feature_34 - only 15 records are null. Retain
- Feature_35 - only 15 records are null. Retain
- Feature_36 - 5683 null values. Need to decide
- Feature_37 - 5683 null values. Need to decide
- Feature_38 - 5683 null values. Need to decide
- Feature_39 - only 15 records are null. Retain
- Feature_40 - only 15 records are null. Retain
- Feature_41 - only 15 records are null. Retain
- Feature_42 - only 15 records are null. Retain
- Feature_43 - only 15 records are null. Retain
- Feature_44 - only 15 records are null. Retain
- Feature_45 - official Email address. Not in full address. can be dropped
- Feature_46 - 95 records empty. Retain
- Feature_47 - No null values. Seems to be masked value of PAN. can be skipped.
- Feature_48 - 18400 null values. Need to decide
- Feature_49 - 99 % data are empty. can be dropped
- Feature_50 - only 15 records are null. Retain
- Feature_51 - 11422 null values. Dropped
- Feature_52 - only 15 records are null. Retain
- Feature_53 - 11511 null values. Dropped
- Feature_54 - only 15 records are null. only 1 unique date is present. Dropped.
- Feature_55 - only 15 records are null. Retain
- Feature_56 - only 15 records are null. Retain
- Feature_57 - 21503 null values. Dropped
- Feature_58 - only 15 records are null. Retain
- Feature_59 - only 15 records are null. Retain
- Feature_60 - only 15 records are null. Retain
- Feature_61 - 23887 null values. Dropped
- Feature_62 - only 15 records are null. Retain
- Feature_63 - only 15 records are null. Data resembles datetime type but not in proper format. Some value contains month as 0. It is difficult to convert these value. Hence it can be dropped.
- Feature_64 - only 15 records are null. Retain
- Feature_65 - only 15 records are null. Retain
- Feature_66 - only 15 records are null. Retain
- Feature_67 - only 15 records are null. Retain
- Feature_68 - only 15 records are null. Retain
- Feature_69 - only 15 records are null. Retain
- Feature_70 - only 15 records are null. But many dates are not in proper format. Dropped.
- Feature_71 - only 15 records are null. Retain
- Feature_72 - only 15 records are null. Retain
- Feature_73 - 20952 null values. Dropped
- Feature_74 - 23880 null values. Dropped
- Feature_75 - only 15 records are null. But many dates are not in proper format. Dropped.
- Feature_76 - only 15 records are null. Retain
- Feature_77 - Masked mobile no. No use. Dropped
- Feature_78 - only 15 records are null. Retain
- Feature_79 - only 15 records are null. Retain

## **5.6 Dropping Unneccesary Columns**

In [ ]:
drop_cols = ['feature_2','feature_4','feature_5','feature_6','feature_10','feature_13','feature_14','feature_15','feature_16','feature_17','feature_18','feature_20','feature_22','feature_24','feature_45','feature_47','feature_48','feature_49','feature_51','feature_53','feature_54','feature_57','feature_61','feature_63','feature_70','feature_73','feature_74','feature_75','feature_77']
Cust_Demographics.drop(drop_cols,axis=1,inplace=True)

In [ ]:
# Cust_Demographics.loc[Cust_Demographics.feature_3 == '','feature_3'] = 0
# Cust_Demographics.feature_3 = Cust_Demographics.feature_3.astype('int')
# Cust_Demographics.feature_7 = Cust_Demographics.feature_7.astype('int')
# Cust_Demographics.loc[Cust_Demographics.feature_8 == '','feature_8'] = 'Not Applicable'
# Cust_Demographics.loc[Cust_Demographics.feature_9 == '','feature_9'] = 'Not Applicable'
# Cust_Demographics.feature_19 = Cust_Demographics.feature_19.astype('int')
# Cust_Demographics.feature_21 = Cust_Demographics.feature_21.astype('datetime64[ns]')
# Cust_Demographics.feature_25 = Cust_Demographics.feature_25.astype('int')
# Cust_Demographics.feature_26 = Cust_Demographics.feature_26.astype('int')

# Cust_Demographics.loc[Cust_Demographics.feature_27 == '','feature_27'] = 'Not Applicable'

# Cust_Demographics.feature_29 = Cust_Demographics.feature_29.astype('int')
# Cust_Demographics.feature_30 = Cust_Demographics.feature_30.astype('int')
# Cust_Demographics.feature_31 = Cust_Demographics.feature_31.astype('int')
# Cust_Demographics.feature_34 = Cust_Demographics.feature_34.astype('int')
# Cust_Demographics.feature_35 = Cust_Demographics.feature_35.astype('int')


# Cust_Demographics.loc[Cust_Demographics.feature_36 == '','feature_36'] = 'Not Applicable'
# Cust_Demographics.loc[Cust_Demographics.feature_37 == '','feature_37'] = 'Not Applicable'
# Cust_Demographics.loc[Cust_Demographics.feature_38 == '','feature_38'] = 'Not Applicable'

# Cust_Demographics.feature_39 = Cust_Demographics.feature_39.astype('int')
# Cust_Demographics.feature_40 = Cust_Demographics.feature_40.astype('int')
# Cust_Demographics.feature_41 = Cust_Demographics.feature_41.astype('int')
# Cust_Demographics.feature_42 = Cust_Demographics.feature_42.astype('int')
# Cust_Demographics.feature_44 = Cust_Demographics.feature_44.astype('int')

# Cust_Demographics.loc[Cust_Demographics.feature_46 == '','feature_46'] = 'Not Applicable'

# Cust_Demographics.feature_52 = Cust_Demographics.feature_52.astype('int')
# Cust_Demographics.feature_55 = Cust_Demographics.feature_55.astype('int')
# Cust_Demographics.feature_56 = Cust_Demographics.feature_56.astype('int')
# Cust_Demographics.feature_64 = Cust_Demographics.feature_64.astype('int')
# Cust_Demographics.feature_65 = Cust_Demographics.feature_65.astype('int')
# Cust_Demographics.feature_66 = Cust_Demographics.feature_66.astype('int')
# Cust_Demographics.feature_67 = Cust_Demographics.feature_67.astype('int')
# Cust_Demographics.feature_68 = Cust_Demographics.feature_68.astype('int')
# Cust_Demographics.feature_69 = Cust_Demographics.feature_69.astype('int')
# Cust_Demographics.feature_71 = Cust_Demographics.feature_71.astype('int')
# Cust_Demographics.feature_76 = Cust_Demographics.feature_76.astype('int')
# Cust_Demographics.feature_78 = Cust_Demographics.feature_78.astype('int')

In [ ]:
Cust_Demographics.loc[Cust_Demographics.feature_3 == '','feature_3'] = 0
Cust_Demographics.feature_21 = Cust_Demographics.feature_21.astype('datetime64[ns]')

## **5.7 Handle Null Values**

In [ ]:
nullcolumn = ['feature_8','feature_9','feature_27','feature_36','feature_37','feature_38','feature_46']
def handlenull(nullcolumn):
    for i in nullcolumn:
        Cust_Demographics.loc[Cust_Demographics[i] == '',i] = 'Not Applicable'
handlenull(nullcolumn)

## **5.8 Encoding**

### **5.8.1 One hot encoding**

In [ ]:
column = ['feature_11','feature_23','feature_33','feature_50','feature_58','feature_59','feature_60','feature_62','feature_72','feature_79']
def OHE(column):
    for i in column:
        enc = OneHotEncoder(sparse=False,drop='first',dtype=int)
        Cust_Demographics[i] = enc.fit_transform(pd.DataFrame(Cust_Demographics[i]))

In [ ]:
OHE(column)

### **5.8.2 Label encoding**

In [ ]:
lblcol = ['feature_1','feature_8','feature_9','feature_12','feature_27','feature_28','feature_32','feature_36','feature_37','feature_38','feature_43','feature_46']
def lblencoding(lblcol):
    for i in lblcol:
        lblenc = LabelEncoder()
        Cust_Demographics[i] = lblenc.fit_transform(pd.DataFrame(Cust_Demographics[i]))

In [ ]:
lblencoding(lblcol)

## **5.9 Removing Duplicated Rows**

In [ ]:
Cust_Demographics.drop_duplicates(inplace=True)

## **5.10 Reducing Dataset size**

In [ ]:
numcolumn = ['feature_3','feature_7','feature_19','feature_25','feature_26','feature_29','feature_30','feature_31','feature_34','feature_35','feature_39','feature_40','feature_41','feature_42','feature_44','feature_52','feature_55','feature_56','feature_64','feature_65','feature_66','feature_67','feature_68','feature_69','feature_71','feature_76','feature_78']
def convert2num(numcolumn):
    for i in numcolumn:
        Cust_Demographics[i] = Cust_Demographics[i].astype('int')

In [ ]:
convert2num(numcolumn)

In [ ]:
for column_name in Cust_Demographics:
    if Cust_Demographics[column_name].dtype == 'int32':
        Cust_Demographics[column_name] = pd.to_numeric(Cust_Demographics[column_name], downcast='integer')

- In the original dataset, memory consumed is 15.1 MB. Processing time will be more if we process with this huge datset size.So before going into processing, we are reducing the memory consumed by the dataset.
- Before reducing -15.1 MB
- After reducing - 2.5 MB

In [ ]:
Cust_Demographics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23881 entries, 0 to 23895
Data columns (total 54 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   dt_opened    23881 non-null  datetime64[ns]
 1   customer_no  23881 non-null  int16         
 2   entry_time   23881 non-null  datetime64[ns]
 3   feature_1    23881 non-null  int8          
 4   feature_3    23881 non-null  int16         
 5   feature_7    23881 non-null  int32         
 6   feature_8    23881 non-null  int8          
 7   feature_9    23881 non-null  int8          
 8   feature_11   23881 non-null  int8          
 9   feature_12   23881 non-null  int8          
 10  feature_19   23881 non-null  int8          
 11  feature_21   23881 non-null  datetime64[ns]
 12  feature_23   23881 non-null  int8          
 13  feature_25   23881 non-null  int8          
 14  feature_26   23881 non-null  int8          
 15  feature_27   23881 non-null  int8          
 16  feature_2

In [ ]:
Cust_Demographics

,dt_opened,customer_no,entry_time,feature_1,feature_3,feature_7,feature_8,feature_9,feature_11,feature_12,feature_19,feature_21,feature_23,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_46,feature_50,feature_52,feature_55,feature_56,feature_58,feature_59,feature_60,feature_62,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_71,feature_72,feature_76,feature_78,feature_79,Bad_label
0,2015-04-18,1,2015-04-13,1,650,500000,0,15,1,3,1,2065-09-03,0,1,2,7,39,400610,1965,0,4,1,2,90000,2,7,2957,0,0,0,0,43,400059,3,1,0,1,21,1,1,0,1,21,15,400610,0,2,90000,21,1,0,1,0,0
1,2015-04-21,2,2015-04-21,1,760,1200000,0,15,1,3,1,2062-07-14,0,1,2,7,9,560042,1969,0,4,1,2,1,2,7,2957,0,0,0,0,6,560042,3,0,0,1,17,1,1,0,1,17,12,560042,0,2,1,17,1,0,1,0,0
2,2015-04-22,3,2015-04-21,1,774,700000,0,15,1,3,2,2066-04-10,0,1,0,7,9,560042,1966,0,4,1,2,1,2,7,2957,0,0,0,0,6,560042,3,0,0,3,17,0,1,0,1,17,12,560042,0,2,1,17,1,0,1,0,0
3,2015-04-25,4,2015-04-15,1,770,500000,0,15,1,3,1,2064-04-16,0,1,3,7,45,411001,1988,0,4,1,2,100000,2,7,2957,0,0,0,0,51,411026,3,1,0,1,21,1,1,0,1,21,16,411001,0,2,100000,21,1,1,1,0,0
4,2015-05-06,5,2015-04-30,1,0,500000,0,15,1,3,1,2054-01-03,0,1,3,7,23,122009,1995,0,4,1,2,300000,2,7,2957,0,0,0,0,28,122009,3,0,0,1,13,1,1,0,1,13,3,122009,0,2,300000,13,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23891,2015-12-31,23892,2015-12-10,3,704,119000,0,15,1,7,1,1988-10-12,0,2,0,5,42,110045,1988,10,2,1,1,33000,7,8,4077,2015,6,0,6,28,122001,3,1,99000,1,10,0,1,0,1,10,157,110045,0,1,33000,13,1,0,1,0,0
23892,2015-12-17,23893,2015-11-16,3,0,35000,0,15,1,7,1,1987-02-01,0,2,0,5,42,110037,2014,12,3,0,1,1,7,8,1678,2012,12,3,0,47,110037,3,0,0,1,10,0,1,0,1,33,102,282001,1,1,1,10,0,0,1,0,0
23893,2015-11-04,23894,2015-09-24,4,748,68000,0,15,1,10,1,1986-11-08,1,2,0,5,42,110033,1986,0,2,1,1,1,7,2,3917,2015,1,3,0,28,122016,3,1,52000,1,10,0,1,0,1,10,157,110033,0,1,1,13,1,0,1,0,0
23894,2015-11-03,23895,2015-09-30,3,740,158000,0,15,1,7,1,1982-11-24,0,1,0,5,42,110095,2010,0,4,1,2,25000,2,7,2957,0,0,0,0,47,110095,3,1,65000,2,10,0,1,0,1,10,157,110095,0,2,25000,10,1,0,1,0,0


In [ ]:
file_name = 'Cust_Demographics_processed.csv'
Cust_Demographics.to_csv(file_name,index=False)

# **6. EDA - Cust_Enquiry**

In [ ]:
query4 = 'select * from Cust_Enquiry'
Cust_Enquiry = fetchresult(query4)

In [ ]:
Cust_Enquiry

,dt_opened,customer_no,upload_dt,enquiry_dt,enq_purpose,enq_amt
0,18-Apr-15,1,21-Apr-15,19-Dec-14,2,3500000
1,18-Apr-15,1,21-Apr-15,05-Mar-14,5,500000
2,18-Apr-15,1,21-Apr-15,05-Mar-14,0,50000
3,18-Apr-15,1,21-Apr-15,22-Feb-14,10,50000
4,18-Apr-15,1,21-Apr-15,11-Jun-13,10,1000
...,...,...,...,...,...,...
413183,04-Dec-15,23896,09-Nov-15,22-Oct-11,5,200000
413184,04-Dec-15,23896,09-Nov-15,06-Jan-11,10,50000
413185,04-Dec-15,23896,09-Nov-15,01-Apr-10,10,50000
413186,04-Dec-15,23896,09-Nov-15,21-Feb-10,1,330000


In [ ]:
Cust_Enquiry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413188 entries, 0 to 413187
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   dt_opened    413188 non-null  object
 1   customer_no  413188 non-null  object
 2   upload_dt    413188 non-null  object
 3   enquiry_dt   413188 non-null  object
 4   enq_purpose  413188 non-null  object
 5   enq_amt      413188 non-null  object
dtypes: object(6)
memory usage: 18.9+ MB


## **6.1 dt_opened**

In [ ]:
Cust_Enquiry[Cust_Enquiry.dt_opened.isnull()]

,dt_opened,customer_no,upload_dt,enquiry_dt,enq_purpose,enq_amt


In [ ]:
Cust_Enquiry.dt_opened = Cust_Enquiry.dt_opened.astype('datetime64[ns]')

#### Inference:
- No null value present.
- Field type changed to datetime64
- Date ranges from 16-Apr-2015 to 31-Dec-2015

## **6.2 customer_no**

In [ ]:
Cust_Enquiry.customer_no.value_counts()

customer_no
10076    308
6775     254
6147     246
11922    214
12736    176
        ... 
23429      1
6891       1
12971      1
19588      1
4666       1
Name: count, Length: 23896, dtype: int64

In [ ]:
Cust_Enquiry[Cust_Enquiry.customer_no.isnull()]

,dt_opened,customer_no,upload_dt,enquiry_dt,enq_purpose,enq_amt


In [ ]:
Cust_Enquiry.customer_no = Cust_Enquiry.customer_no.astype('int')

#### Inference:
- No null value present in the customer number column
- Customer no - 10076 has enquired 308 times
- Field type changed to integer.

## **6.3 upload_dt**

In [ ]:
Cust_Enquiry.upload_dt = Cust_Enquiry.upload_dt.astype('datetime64[ns]')

In [ ]:
drop_rows = Cust_Enquiry.loc[Cust_Enquiry.upload_dt.isnull(),'upload_dt'].index

In [ ]:
Cust_Enquiry.drop(drop_rows,axis=0,inplace=True)

#### Inference:
- 110 null value present.
- Field type changed to datetime64
- Date ranges from 21-Apr-2015 to 31-Dec-2015

## **6.4 enquiry_dt**

In [ ]:
Cust_Enquiry.enquiry_dt = Cust_Enquiry.enquiry_dt.astype('datetime64[ns]')

In [ ]:
Cust_Enquiry[Cust_Enquiry.enquiry_dt.isnull()]

,dt_opened,customer_no,upload_dt,enquiry_dt,enq_purpose,enq_amt


#### Inference:
- Field type changed to datetime64

## **6.5 enq_purpose**

In [ ]:
Cust_Enquiry.enq_purpose = Cust_Enquiry.enq_purpose.astype('int')

In [ ]:
Cust_Enquiry[Cust_Enquiry.enq_purpose.isnull()]

,dt_opened,customer_no,upload_dt,enquiry_dt,enq_purpose,enq_amt


In [ ]:
Cust_Enquiry.enq_purpose.value_counts()

enq_purpose
10    238150
5      81881
1      23708
6      17426
2      15576
0      15048
13     11179
51      3942
3       1728
7       1351
12      1000
8        563
17       410
4        280
32       170
54       154
52       130
14        95
53        72
9         51
40        22
35        19
59        15
33        14
56        13
15        13
11        13
55        12
57        10
41         7
43         6
58         6
36         6
34         3
31         3
16         2
Name: count, dtype: int64

#### Inference:
- No null value present.
- Field type changed to int.
- Enquiry type 10 was done max times of 238150.

## **6.6 enq_amt**

In [ ]:
Cust_Enquiry.enq_amt = Cust_Enquiry.enq_amt.astype('int')

In [ ]:
Cust_Enquiry[Cust_Enquiry.enq_amt.isnull()]

,dt_opened,customer_no,upload_dt,enquiry_dt,enq_purpose,enq_amt


In [ ]:
Cust_Enquiry.enq_amt.value_counts()

enq_amt
50000     79949
10000     54584
1000      50162
100000    38876
15000     15442
          ...  
48820         1
127735        1
144692        1
579255        1
50144         1
Name: count, Length: 7383, dtype: int64

#### Inference:
- No null value present.
- Field type changed to int.
- Enquiry amount 50000 was enquired max times of 79949.

## **6.7 Removing Duplicated Rows**

In [ ]:
Cust_Enquiry.drop_duplicates(inplace=True)

## **6.8 Reducing Dataset size**

In [ ]:
for column_name in Cust_Enquiry:
    if Cust_Enquiry[column_name].dtype == 'int32':
        Cust_Enquiry[column_name] = pd.to_numeric(Cust_Enquiry[column_name], downcast='integer')

- In the original dataset, memory consumed is 19.1 MB. Processing time will be more if we process with this huge datset size.So before going into processing, we are reducing the memory consumed by the dataset.
- Before reducing -19.1 MB
- After reducing - 15.4 MB

## **6.9 Extracting Enquiry count**

In [ ]:
# Since this table holds information about the customer enquiry information, other fields can be skipped and new field with enquiry_count can be created and this data can be merged with other 2 tables.
cust_enquiries = Cust_Enquiry.groupby('customer_no').size().reset_index(name='Enquiry_Count')
cust_enquiries

,customer_no,Enquiry_Count
0,1,18
1,2,67
2,3,1
3,4,33
4,5,2
...,...,...
23782,23892,16
23783,23893,12
23784,23894,4
23785,23895,8


In [ ]:
file_name = 'Cust_Enquiry_processed.csv'
Cust_Enquiry.to_csv(file_name,index=False)

# **7.0 Merging the tables**

In [ ]:
# Merging the processed data of Cust_Account, Cust_Demographics & cust_enquiries table.
merged_data = pd.merge(Cust_Account,Cust_Demographics,on='customer_no',how='inner')
data = pd.merge(merged_data,cust_enquiries,on='customer_no',how='inner')
data

,dt_opened_x,customer_no,upload_dt,acct_type,opened_dt,reporting_dt,high_credit_amt,cur_balance_amt,paymt_str_dt,paymt_end_dt,dt_opened_y,entry_time,feature_1,feature_3,feature_7,feature_8,feature_9,feature_11,feature_12,feature_19,feature_21,feature_23,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_46,feature_50,feature_52,feature_55,feature_56,feature_58,feature_59,feature_60,feature_62,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_71,feature_72,feature_76,feature_78,feature_79,Bad_label,Enquiry_Count
0,2015-11-10,12265,2015-10-20,6,2013-06-09,2015-09-30,20900,0,2015-09-01,2014-07-01,2015-11-10,2015-10-15,4,683,56000,0,15,1,10,1,1984-06-15,0,1,0,6,42,110059,2006,0,2,1,1,35000,7,8,4510,2009,1,4,0,28,122001,3,0,0,1,10,0,1,0,1,10,157,110059,0,1,35000,13,1,0,1,0,1,23
1,2015-11-10,12265,2015-10-20,10,2012-05-25,2015-10-03,16201,10390,2015-10-01,2012-11-01,2015-11-10,2015-10-15,4,683,56000,0,15,1,10,1,1984-06-15,0,1,0,6,42,110059,2006,0,2,1,1,35000,7,8,4510,2009,1,4,0,28,122001,3,0,0,1,10,0,1,0,1,10,157,110059,0,1,35000,13,1,0,1,0,1,23
2,2015-11-10,12265,2015-10-20,10,2012-03-22,2015-09-30,41028,34420,2015-09-01,2012-10-01,2015-11-10,2015-10-15,4,683,56000,0,15,1,10,1,1984-06-15,0,1,0,6,42,110059,2006,0,2,1,1,35000,7,8,4510,2009,1,4,0,28,122001,3,0,0,1,10,0,1,0,1,10,157,110059,0,1,35000,13,1,0,1,0,1,23
3,2015-07-20,15606,2015-07-09,10,2006-01-13,2009-01-31,93473,0,2007-07-01,2006-02-01,2015-07-20,2015-07-06,3,687,125000,0,15,1,7,1,1980-06-20,0,1,2,9,18,121003,2009,0,4,1,1,100000,7,1,2019,2013,1,2,0,21,121003,3,0,0,1,13,0,1,0,1,13,39,121003,0,1,100000,13,1,0,1,0,1,33
4,2015-07-20,15606,2015-07-09,6,2015-01-18,2015-05-31,20250,13500,2015-05-01,2015-01-01,2015-07-20,2015-07-06,3,687,125000,0,15,1,7,1,1980-06-20,0,1,2,9,18,121003,2009,0,4,1,1,100000,7,1,2019,2013,1,2,0,21,121003,3,0,0,1,13,0,1,0,1,13,39,121003,0,1,100000,13,1,0,1,0,1,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182704,2015-06-11,1611,2015-05-25,10,2007-05-02,2015-04-30,24228,24228,2015-04-01,2012-05-01,2015-06-11,2015-05-21,6,746,138000,0,15,1,14,1,1976-02-10,0,1,3,9,25,500017,1995,0,4,1,1,45800,7,1,2629,1999,0,11,0,31,500003,3,0,0,1,2,0,1,0,1,2,14,500017,0,1,45800,2,1,0,1,0,0,19
182705,2015-06-11,1611,2015-05-25,10,2007-04-11,2015-04-30,19584,0,2010-04-01,2009-08-01,2015-06-11,2015-05-21,6,746,138000,0,15,1,14,1,1976-02-10,0,1,3,9,25,500017,1995,0,4,1,1,45800,7,1,2629,1999,0,11,0,31,500003,3,0,0,1,2,0,1,0,1,2,14,500017,0,1,45800,2,1,0,1,0,0,19
182706,2015-06-11,1611,2015-05-25,10,2006-11-20,2013-12-31,50844,0,2012-04-01,2010-12-01,2015-06-11,2015-05-21,6,746,138000,0,15,1,14,1,1976-02-10,0,1,3,9,25,500017,1995,0,4,1,1,45800,7,1,2629,1999,0,11,0,31,500003,3,0,0,1,2,0,1,0,1,2,14,500017,0,1,45800,2,1,0,1,0,0,19
182707,2015-06-11,1611,2015-05-25,10,2006-10-17,2015-04-15,35587,0,2012-03-01,2009-04-01,2015-06-11,2015-05-21,6,746,138000,0,15,1,14,1,1976-02-10,0,1,3,9,25,500017,1995,0,4,1,1,45800,7,1,2629,1999,0,11,0,31,500003,3,0,0,1,2,0,1,0,1,2,14,500017,0,1,45800,2,1,0,1,0,0,19


# **8.0 Dropping Duplicated Rows**

In [ ]:
data.drop_duplicates(inplace=True)

# **9.0 Reducing dataset memory**

In [ ]:
for column_name in data:
    if data[column_name].dtype == 'int64':
        data[column_name] = pd.to_numeric(data[column_name], downcast='integer')

# **10.0 Saving the final version dataset**

In [ ]:
file_name = 'final_data.csv'
data.to_csv(file_name,index=False)

# **11.0 Conclusion:**

- In this preprocessing, we have fetched 3 table records from the database, processed and merged into 1 csv file. This merged file will be used for Modelling different algorithms.
- New file contains 182709 rows & 64 columns.
    - Features - 63
    - Target - 1 (Bad_Label)
- Preprocessing done in this file:
    - NULL values handled
    - Type conversion & dataset memory optimized.
    - Encoding applied.
    - Final processed csv file generated with a size of 28.1 MB